In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import scipy.stats as stats
import sys

sys.path.append("../")
import vuong_tests_fast

In [2]:
class OLS_loglike(GenericLikelihoodModel):
    
    def __init__(self, *args,ols=False, **kwargs):
        super(OLS_loglike,self).__init__(*args,**kwargs)
        self.ols = ols

    def loglikeobs(self, params):
        y = self.endog
        x = self.exog
        mu_y = np.matmul(x,params)  
        resid = y - mu_y
        sigma = np.sqrt(np.sum(resid**2)/resid.shape[0])
        pr_y = stats.norm.logpdf( resid, loc=0,scale=sigma )
        return pr_y


def setup_shi(yn,xn,return_model=False,num_params=4):
    x1n,x2n = xn[:,0],xn[:,1:num_params+1]
    
    # model 1 grad, etc.
    model1 = sm.OLS(yn,sm.add_constant(x1n))
    model1_fit = model1.fit(disp=False)
    params1 = (model1_fit.params)
    
    model1_deriv = OLS_loglike(yn,sm.add_constant(x1n))
    ll1 = model1_deriv.loglikeobs(model1_fit.params)
    grad1 =  model1_deriv.score_obs(model1_fit.params)    
    hess1 = model1_deriv.hessian(model1_fit.params)
    
    #model 2 grad, etc.
    model2 = sm.OLS(yn,sm.add_constant(x2n))
    model2_fit = model2.fit(disp=False)
    params2 = (model2_fit.params)
    
    model2_deriv = OLS_loglike(yn,sm.add_constant(x2n))
    ll2 = model2_deriv.loglikeobs(model2_fit.params)
    grad2 =  model2_deriv.score_obs(model2_fit.params)    
    hess2 = model2_deriv.hessian(model2_fit.params)
    
    if return_model:
        return ll1,grad1,hess1,params1,model1,ll2,grad2,hess2,params2,model2
    return ll1,grad1,hess1,params1,ll2,grad2,hess2,params2


In [3]:

def gen_data(nobs=1000, a=0.25, num_params=4):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a*x[:,0] + a/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn,return_model=False,num_params=15)
print(grad1.shape,hess1.shape)

(1000, 2) (2, 2)


In [4]:
num_sims = 500
trials =1000

c1 = .005
c2 = .0025

adapt_c = False

# a  = .25, k= 9

In [5]:
a = 0.25
num_params=9


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.96 & 0.97 & 0.92 & 0.95 & 0.98   \\
Model 1 & 0.03 & 0.03 & 0.03 & 0.08 & 0.05 & 0.02   \\
Model 2 & 0.01 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.96 , 0.034, 0.006]), array([0.96 , 0.034, 0.006]), array([0.968, 0.03 , 0.002]), array([0.92 , 0.078, 0.002]), array([0.948, 0.052, 0.   ]), array([0.98, 0.02, 0.  ]), 2.557511986006458, 4.709247581184681, 4.925392621574783)


In [7]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.83 & 0.83 & 0.84 & 0.77 & 0.82 & 0.89   \\
Model 1 & 0.16 & 0.16 & 0.16 & 0.23 & 0.18 & 0.11   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.832, 0.164, 0.004]), array([0.832, 0.164, 0.004]), array([0.838, 0.16 , 0.002]), array([0.772, 0.228, 0.   ]), array([0.822, 0.178, 0.   ]), array([0.894, 0.106, 0.   ]), 6.911770731821634, 6.722741771572422, 6.647047371919158)


In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=100, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.98 & 0.99 & 0.99 & 0.98 & 0.98 & 1.00   \\
Model 1 & 0.01 & 0.00 & 0.00 & 0.02 & 0.02 & 0.00   \\
Model 2 & 0.01 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.984, 0.01 , 0.006]), array([0.994, 0.004, 0.002]), array([0.992, 0.002, 0.006]), array([0.976, 0.024, 0.   ]), array([0.978, 0.022, 0.   ]), array([0.998, 0.002, 0.   ]), -0.012752202864774399, 3.1850718342354636, 3.3773908197560814)


# a  = .25, k= 4, something not right?

In [9]:
a = 0.25
num_params=4


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.95 & 0.96 & 0.96 & 0.97 & 0.99   \\
Model 1 & 0.02 & 0.02 & 0.01 & 0.03 & 0.02 & 0.01   \\
Model 2 & 0.03 & 0.03 & 0.03 & 0.02 & 0.01 & 0.01   \\
\hline
\end{tabular}
(array([0.95 , 0.016, 0.034]), array([0.95 , 0.016, 0.034]), array([0.964, 0.01 , 0.026]), array([0.956, 0.026, 0.018]), array([0.97 , 0.016, 0.014]), array([0.986, 0.008, 0.006]), -1.1127199521756006, 5.739166291234276, 5.619994660699458)


# a  = .25, K = 19

In [11]:
a = 0.25
num_params=19


In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, a=a,  num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.88 & 0.88 & 0.90 & 0.79 & 0.85 & 0.95   \\
Model 1 & 0.11 & 0.11 & 0.09 & 0.21 & 0.15 & 0.05   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.884, 0.114, 0.002]), array([0.884, 0.114, 0.002]), array([0.904, 0.094, 0.002]), array([0.79, 0.21, 0.  ]), array([0.852, 0.148, 0.   ]), array([0.948, 0.052, 0.   ]), 4.292232150586109, 4.559987238381474, 4.534904970070584)


# evidence of power

In [13]:
a1,a2 = np.sqrt(1.09-1), 0.00
num_params= 9


def gen_data2(nobs=1000, a1=np.sqrt(1.09-1), a2=0.00 , num_params=19):
    x = np.random.normal(scale=1., size=(nobs,1+num_params))
    e = np.random.normal(loc=0.0, scale=1.0, size=nobs)
    y = 1 + a1*x[:,0] + a2/np.sqrt(num_params)*x[:,1:num_params+1].sum(axis=1) + e
    return y,x,nobs

In [14]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.60 & 0.60 & 0.63 & 0.47 & 0.55 & 0.76   \\
Model 1 & 0.40 & 0.40 & 0.37 & 0.53 & 0.45 & 0.24   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.604, 0.396, 0.   ]), array([0.604, 0.396, 0.   ]), array([0.628, 0.372, 0.   ]), array([0.47, 0.53, 0.  ]), array([0.552, 0.448, 0.   ]), array([0.758, 0.242, 0.   ]), 9.046384124458335, 4.947842100245946, 4.850747929265023)


In [15]:
a1,a2 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.55 & 0.57 & 0.37 & 0.53 & 0.73   \\
Model 1 & 0.45 & 0.45 & 0.43 & 0.63 & 0.47 & 0.27   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.554, 0.446, 0.   ]), array([0.554, 0.446, 0.   ]), array([0.57, 0.43, 0.  ]), array([0.366, 0.634, 0.   ]), array([0.526, 0.474, 0.   ]), array([0.73, 0.27, 0.  ]), 9.474335592196773, 5.017137686135558, 5.001451733735219)


In [16]:
a1,a2 = np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.55 & 0.57 & 0.63 & 0.49 & 0.55 & 0.76   \\
Model 1 & 0.45 & 0.43 & 0.37 & 0.51 & 0.45 & 0.24   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.548, 0.452, 0.   ]), array([0.568, 0.432, 0.   ]), array([0.632, 0.368, 0.   ]), array([0.488, 0.512, 0.   ]), array([0.548, 0.452, 0.   ]), array([0.758, 0.242, 0.   ]), 9.231888469989613, 4.623522223670925, 4.650056429548605)


In [17]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.56 & 0.56 & 0.62 & 0.44 & 0.54 & 0.74   \\
Model 1 & 0.44 & 0.44 & 0.38 & 0.56 & 0.46 & 0.26   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.56, 0.44, 0.  ]), array([0.56, 0.44, 0.  ]), array([0.62, 0.38, 0.  ]), array([0.438, 0.562, 0.   ]), array([0.536, 0.464, 0.   ]), array([0.74, 0.26, 0.  ]), 9.263277674326453, 4.834977753802538, 4.953040250618679)


In [18]:
a1,a2 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.57 & 0.57 & 0.62 & 0.42 & 0.53 & 0.73   \\
Model 1 & 0.43 & 0.43 & 0.38 & 0.58 & 0.47 & 0.27   \\
Model 2 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
\hline
\end{tabular}
(array([0.574, 0.426, 0.   ]), array([0.574, 0.426, 0.   ]), array([0.616, 0.384, 0.   ]), array([0.418, 0.582, 0.   ]), array([0.534, 0.466, 0.   ]), array([0.73, 0.27, 0.  ]), 9.297831016993339, 4.6422233631336045, 4.885075718329856)


# evidence of power 2

In [19]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.72 & 0.73 & 0.76 & 0.92 & 0.96 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.28 & 0.27 & 0.24 & 0.08 & 0.04 & 0.01   \\
\hline
\end{tabular}
(array([0.72, 0.  , 0.28]), array([0.728, 0.   , 0.272]), array([0.756, 0.   , 0.244]), array([0.918, 0.   , 0.082]), array([0.956, 0.   , 0.044]), array([0.992, 0.   , 0.008]), -6.058317991013257, 3.2149250722118707, 3.5688437487334492)


In [20]:
a2,a1 = np.sqrt(1.09**.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=500, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.74 & 0.74 & 0.75 & 0.90 & 0.95 & 0.99   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.26 & 0.26 & 0.25 & 0.10 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.742, 0.   , 0.258]), array([0.742, 0.   , 0.258]), array([0.75, 0.  , 0.25]), array([0.902, 0.002, 0.096]), array([0.954, 0.   , 0.046]), array([0.994, 0.   , 0.006]), -6.05446185276289, 3.4226233301629367, 3.5610397993075766)


In [21]:
a2,a1 =np.sqrt(1.09**2.5-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=100, a1=a1, a2=a2, num_params=num_params)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.69 & 0.74 & 0.78 & 0.89 & 0.91 & 0.98   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.31 & 0.26 & 0.22 & 0.11 & 0.09 & 0.02   \\
\hline
\end{tabular}
(array([0.69, 0.  , 0.31]), array([0.744, 0.   , 0.256]), array([0.778, 0.   , 0.222]), array([0.89, 0.  , 0.11]), array([0.91, 0.  , 0.09]), array([0.982, 0.   , 0.018]), -6.096095307615932, 3.586970648399912, 3.4610702943938896)


In [22]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=4)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.20 & 0.20 & 0.22 & 0.43 & 0.53 & 0.74   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.80 & 0.80 & 0.78 & 0.57 & 0.47 & 0.26   \\
\hline
\end{tabular}
(array([0.196, 0.   , 0.804]), array([0.2, 0. , 0.8]), array([0.224, 0.   , 0.776]), array([0.432, 0.   , 0.568]), array([0.53, 0.  , 0.47]), array([0.74, 0.  , 0.26]), -12.449825508154793, 4.962302246288449, 4.937558113184435)


In [23]:
a2,a1 = np.sqrt(1.09-1), 0.00
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data2(nobs=250, a1=a1, a2=a2, num_params=19)
mc_out = vuong_tests_fast.monte_carlo(num_sims,gen_data_ex,setup_shi,trials=trials,c1=c1,c2=c2,adapt_c = adapt_c)
vuong_tests_fast.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
Model &  Normal & Two-Step & Bootstrap & Bootstrap-TIC & Bootstrap-ND & Shi (2015) \\ \hline \hline
No selection & 0.92 & 0.93 & 0.94 & 0.98 & 0.99 & 1.00   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.08 & 0.07 & 0.06 & 0.02 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.924, 0.   , 0.076]), array([0.93, 0.  , 0.07]), array([0.942, 0.   , 0.058]), array([0.978, 0.002, 0.02 ]), array([0.992, 0.   , 0.008]), array([0.996, 0.   , 0.004]), -3.710739332981895, 2.695403298467543, 2.9132658471285584)
